<a href="https://colab.research.google.com/github/ankit-rathi/Tradevesting_v1/blob/main/myProspects_200DMA_RSI14_Signals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import datetime
import pytz
tz_NY = pytz.timezone('Asia/Kolkata')
datetime_NY = datetime.datetime.now(tz_NY)
print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

Run date time (IST): 2025-01-24 08:47:01


In [2]:
!pip install yfinance
!pip install pandas-ta
#!pip install talib-binary

In [3]:
import pandas as pd
import numpy as np
#from google.colab import drive
#drive.mount('/content/drive')

#mypf = pd.read_csv('/content/drive/My Drive/data/stocks/myProspectsScrips.csv')

path_mypf = 'https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/'

mypf = pd.read_csv(path_mypf + 'myProspectsScrips.csv')

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df


pps_df = get_mypps_df()
scr_df = get_myscreen_df()

df_stocks = pd.merge(pps_df, scr_df, on = 'Symbol')

df_stocks['Conviction'] = df_stocks['Conviction'] + '-' + df_stocks['CapType']

stock_list = df_stocks[df_stocks['Conviction'].isin(['X-LC','H-LC'])]['Symbol'].values

all_prospects = stock_list

#mypf = mypf[mypf.Forecast.notnull()]
#stock_ids = mypf['Symbol'].unique()

#all_prospects.sort()
#stock_ids

In [4]:
# import necessary libraries

%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from tabulate import tabulate
import warnings
warnings.filterwarnings('ignore')
import pandas_datareader.data as web
import yfinance as yfin
import pandas_ta as ta

In [5]:
# set start and end date
import datetime
start_date = (datetime.date.today() + datetime.timedelta(days=-365*5)).strftime('%Y-%m-%d')
start1_date = (datetime.date.today() + datetime.timedelta(days=-1*365)).strftime('%Y-%m-%d')
end_date = (datetime.date.today() + datetime.timedelta(days=0)).strftime('%Y-%m-%d')

In [10]:
# function to compute stock attributes
def get_common_features():
  def stock_prec_dev(stock_symbol):
      #stock_symbol = 'ULTRACEMCO.NS'
      short_window = 20
      #mid_window = 50
      long_window = 200
      moving_avg = 'WMA'

      # import the closing price data of the stock for the aforementioned period of time in Pandas dataframe
      start = datetime.datetime(*map(int, start_date.split('-')))
      end = datetime.datetime(*map(int, end_date.split('-')))
      stock_df = yfin.Ticker(stock_symbol).history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]
      stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
      stock_df['200_DMA'] = round(stock_df['Close'].rolling(window = long_window, min_periods = 1).mean(),0)
      stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_DMA'])*100/stock_df['200_DMA'],2)
      stock_df.dropna(axis = 0, inplace = True) # remove any null rows

      stock_df['Stock'] = stock_symbol

      stock_df['Close'] = round(stock_df['Close'],2)
      stock_df['Max'] = round(max(stock_df['Close']),2)
      stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start1_date)]
      stock_df['Min'] = round(min(stock_df['Close']),2)
      stock_df['Gained%'] = round((stock_df['Close'] - stock_df['Min'])*100/stock_df['Min'],2)
      stock_df['ATH%'] = round((stock_df['Max'] - stock_df['Close'])*100/stock_df['Close'],2)
      stock_df.ta.rsi(append=True)
      #stock_df.rename(columns={'RSI_14':'RSI'}, inplace=True)
      stock_df['Close'] = round(stock_df['Close'],0)
      stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
      stock_df.drop(['Open', 'Low', 'High', 'Volume'
                    ], axis=1, inplace=True)

      return stock_df

  df_prec_dev = pd.DataFrame()

  for stock_id in all_prospects:
      #print(stock_id)
      stock_id = stock_id.upper() + '.NS'
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      tmp = tmp.tail(1)
      #df_prec_dev = df_prec_dev.append(tmp)
      df_prec_dev = pd.concat([df_prec_dev, tmp])
  return df_prec_dev, stock_prec_dev

df_common_features, stock_prec_dev = get_common_features()


def get_stock_buy_sell(stock_symbol):

    df_prec_dev = stock_prec_dev(stock_symbol)
    df_prec_dev['Signal'] = 0.0
    df_prec_dev['Signal'] = np.where(((df_prec_dev['Close'] < df_prec_dev['200_DMA']) & (df_prec_dev['RSI_14'] < 30) ), 1.0, df_prec_dev['Signal'])

    df_prec_dev = df_prec_dev[df_prec_dev['Signal'] != 0]

    return df_prec_dev.tail(1)

def get_daily_buy_sell():
    df_buy_sell = pd.DataFrame()
    for stock_id in all_prospects:
        df_tmp = get_stock_buy_sell(stock_id+'.NS')
        df_tmp['Conviction'] = df_stocks[(df_stocks['Symbol']) == stock_id.upper()]['Conviction'].values[0]
        df_tmp['InFolio'] = df_stocks[(df_stocks['Symbol']) == stock_id.upper()]['InFolio'].values[0]
        #df_buy_sell = df_buy_sell.append(df_tmp)
        df_buy_sell = pd.concat([df_buy_sell, df_tmp])


    df_buy_sell['Signal'] = df_buy_sell['Signal'].apply(lambda x: 'Buy' if x >= 1 else 'Sell')
    return df_buy_sell.sort_index()

df_daily_buy_sell = get_daily_buy_sell()

In [11]:
df_daily_buy_sell[df_daily_buy_sell['Signal'] == 'Buy'].tail(30)

,Close,200_DMA,Dev%_200,Stock,Max,Min,Gained%,ATH%,RSI_14,Signal,Conviction,InFolio
Date,,,,,,,,,,,,
2024-05-03 00:00:00+05:30,1545.0,1782.0,-13.30,KOTAKBANK.NS,2205.38,1545.01,0.00,42.74,28.0,Buy,X-LC,0.0
2024-05-14 00:00:00+05:30,1298.0,1336.0,-2.84,HCLTECH.NS,1988.42,1280.91,1.34,53.18,27.0,Buy,H-LC,1.0
2024-10-22 00:00:00+05:30,6678.0,6976.0,-4.27,BAJFINANCE.NS,8127.75,6279.73,6.34,21.71,28.0,Buy,X-LC,1.0
2024-10-25 00:00:00+05:30,3326.0,3560.0,-6.56,LT.NS,3947.30,3245.93,2.48,18.67,29.0,Buy,X-LC,NaN
2024-10-30 00:00:00+05:30,1418.0,1486.0,-4.56,CIPLA.NS,1680.50,1306.03,8.59,18.49,24.0,Buy,H-LC,NaN
2024-10-31 00:00:00+05:30,3906.0,3983.0,-1.93,TCS.NS,4471.39,3596.20,8.62,14.47,29.0,Buy,X-LC,1.0
2024-11-04 00:00:00+05:30,11052.0,12015.0,-8.01,MARUTI.NS,13495.60,9788.54,12.91,22.11,28.0,Buy,X-LC,NaN
2024-11-07 00:00:00+05:30,3121.0,3515.0,-11.21,TITAN.NS,3854.04,3120.85,0.00,23.49,25.0,Buy,X-LC,0.0
2024-11-14 00:00:00+05:30,2183.0,2500.0,-12.69,NESTLEIND.NS,2755.50,2149.90,1.53,26.24,26.0,Buy,X-LC,1.0


In [8]:
#tmp_df = df_daily_buy_sell.reset_index()
#tmp_df.to_csv('/content/drive/My Drive/data/stocks/myProspects-Buy-Sell-Signals.csv', index=False)

In [9]:
#stock_symbol = 'INFY.NS'
#plot_stock_buy_sell(stock_symbol)